# Aprimorando a MLP com Grid Search para escolher o melhor número de camadas intermediárias no Mapa de Hénon

## 1. Importando as bibliotecas necessárias

### 1.1 Bibliotecas gerais

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # a biblioteca 'seaborn' contém vários estilos para os gráficos do 'matpĺotlib'

# agora, melhoramos a qualidade de saida e de visualizacao da imagem 
# alem de mudar a fonte padrao para uma do latex
sns.set_style("ticks")
plt.rcParams['savefig.dpi'] = 200
plt.rcParams["figure.dpi"] = 100

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})
plt.style.use('dark_background')

### 1.2 Bibliotecas para MLP

In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')

from tensorflow import keras

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [4]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

### 1.3 Bibliotecas dos sistemas caóticos

In [5]:
import sys 
sys.path.insert(0, '../../../../scripts')

import timeseries as times
import henonmap as hmap

## 2. Gerando a série temporal do mapa de Hénon

Os valores para os parâmetros das equações serão os "básicos" apresentados no *paper* original.

In [6]:
a = 1.4
b = 0.3
x_inicial = 1.0
y_inicial = 0.0
n_inicial = 0

estados = np.array([x_inicial, y_inicial, n_inicial])

mapa = hmap.MapaHenon(estado_inicial=estados)

In [7]:
n_iteracoes = 5000

for n in range(0, n_iteracoes):
    mapa.iterar()
    estados = np.vstack((estados, mapa.ler_estado()))

In [8]:
x = estados[:, 0]
y = estados[:, 1]
n = estados[:, 2]

In [9]:
fig, ax = plt.subplots()
ax.plot(n, x, color='LimeGreen')

ax.set_title("100 primeiras iterações da série temporal em $\hat{x}$ do Mapa de Hénon para\n $a =$ " + str(a) + " e $b =$ " + str(b) + " com $x[0] =$ " + str(x[0]) + " e $y[0] =$ " + str(y[0]))
ax.set_ylabel('$x[n]$')
ax.set_xlabel('$n$')
ax.set_xlim(0, 100)  
ax.grid(True)

fig.tight_layout()
sns.despine()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2.1 Dividindo em um conjunto de treinamento e de teste, para K = 4 e L = 3

In [10]:
K = 4
L = 3
tam_teste = 0.15

In [11]:
serie_temporal = times.SerieTemporal(x, K=K, L=L)

In [13]:
X_treino, X_teste, y_treino, y_teste = serie_temporal.dividir_treino_teste(tam_teste)

## 3. Definindo o modelo para a MLP

### 3.1 Definindo função para criar a MLP

In [49]:
def criar_modelo(input_size, optimizer='Nadam', batch_normalization='OFF', learning_rate=0.001, activation='selu', init_mode='lecun_normal', n_neurons=30,
                 n_hidden_layers=1, loss="mean_squared_error", name="MLP"):
    
    if (optimizer=='Nadam'):
        model_optimizer = keras.optimizers.Nadam()
    
    model_optimizer.learning_rate.assign(learning_rate)
    
    model = keras.Sequential(name="MLP")
    model.add(keras.layers.Dense(input_size, input_dim=input_size, name="camada_de_entrada", activation = 'linear'))
    if (batch_normalization == 'ON'):
        model.add(keras.layers.BatchNormalization(name="camada_de_batch_normalization"))
    for i in range(0, n_hidden_layers):
        model.add(keras.layers.Dense(n_neurons, input_dim=input_size, activation=activation, kernel_initializer=init_mode, name="camada_intermediaria_"+str(i+1)))
    model.add(keras.layers.Dense(1, activation='linear', name="camada_de_saida"))
    
    model.compile(
        optimizer = model_optimizer,
        loss = loss)
    
    model.build()
    return model

### 3.2 Definindo parâmetros que não serão definidos pelo *Grid Search*

In [50]:
loss = "mean_squared_error"
optimizer = 'Nadam'
batch_size = 8
batch_normalization = 'OFF'
activation = 'sigmoid'
init_mode = 'glorot_normal'
n_neurons = 50
learning_rate = 0.003
input_size = K

### 3.3 Executando *Grid Search* para saber o melhor número de camadas intermediárias

In [52]:
model_cv = KerasRegressor(build_fn=criar_modelo, epochs=100, verbose=0, batch_size=batch_size, batch_normalization=batch_normalization,
                          learning_rate=learning_rate, activation=activation, init_mode=init_mode, n_neurons=n_neurons,
                          loss=loss, optimizer=optimizer, input_size=input_size)

In [53]:
param_grid = {'n_hidden_layers' : (1, 2, 3, 4, 5)}

In [54]:
grid = GridSearchCV(estimator=model_cv, param_grid=param_grid, n_jobs=1, cv=4, scoring='neg_mean_squared_error', verbose=1)

In [55]:
grid_result = grid.fit(X_treino, y_treino)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


In [56]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -0.000879 using {'n_hidden_layers': 3}


In [57]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

-0.003577 (0.001562) with: {'n_hidden_layers': 1}
-0.002442 (0.002089) with: {'n_hidden_layers': 2}
-0.000879 (0.000299) with: {'n_hidden_layers': 3}
-0.001615 (0.001482) with: {'n_hidden_layers': 4}
-0.005203 (0.002802) with: {'n_hidden_layers': 5}


### Treino com o melhor modelo

In [66]:
n_hidden_layers=3

In [67]:
model = criar_modelo(name="MLP-Henon", input_size=K, optimizer=optimizer, batch_normalization=batch_normalization, learning_rate=learning_rate, activation=activation,
                     init_mode=init_mode, n_neurons=n_neurons, n_hidden_layers=n_hidden_layers, loss=loss)

In [68]:
model.summary()

Model: "MLP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
camada_de_entrada (Dense)    (None, 4)                 20        
_________________________________________________________________
camada_intermediaria_1 (Dens (None, 50)                250       
_________________________________________________________________
camada_intermediaria_2 (Dens (None, 50)                2550      
_________________________________________________________________
camada_intermediaria_3 (Dens (None, 50)                2550      
_________________________________________________________________
camada_de_saida (Dense)      (None, 1)                 51        
Total params: 5,421
Trainable params: 5,421
Non-trainable params: 0
_________________________________________________________________


In [69]:
X_treino, X_val, y_treino, y_val = train_test_split(X_treino, y_treino, test_size=0.1)

In [70]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, monitor='val_loss')

In [72]:
history = model.fit(X_treino, y_treino, epochs=100,
                            callbacks=early_stopping, validation_data=(X_val, y_val),
                            batch_size=batch_size)
treinamento = pd.DataFrame(history.history)

Epoch 1/100
430/430 [==============================] - 1s 2ms/step - loss: 0.4912 - val_loss: 0.4061
Epoch 2/100
430/430 [==============================] - 1s 2ms/step - loss: 0.4388 - val_loss: 0.3979
Epoch 3/100
430/430 [==============================] - 1s 2ms/step - loss: 0.3983 - val_loss: 0.3745
Epoch 4/100
430/430 [==============================] - 1s 2ms/step - loss: 0.3855 - val_loss: 0.3689
Epoch 5/100
430/430 [==============================] - 1s 2ms/step - loss: 0.3777 - val_loss: 0.3688
Epoch 6/100
430/430 [==============================] - 1s 2ms/step - loss: 0.3672 - val_loss: 0.3621
Epoch 7/100
430/430 [==============================] - 1s 2ms/step - loss: 0.3507 - val_loss: 0.3439
Epoch 8/100
430/430 [==============================] - 1s 2ms/step - loss: 0.3539 - val_loss: 0.3800
Epoch 9/100
430/430 [==============================] - 1s 2ms/step - loss: 0.3339 - val_loss: 0.3215
Epoch 10/100
430/430 [==============================] - 1s 2ms/step - loss: 0.3089 - val_lo

## Teste com o melhor modelo

In [73]:
y_pred = model.predict(X_teste)

### Gráfico

In [74]:
fig, ax = plt.subplots()
ax.plot(n[len(n)-int(len(n)*tam_teste):,], y_teste, color='LimeGreen', label='Valor real')
ax.plot(n[len(n)-int(len(n)*tam_teste):,], y_pred, color='Silver', label='MLP')

ax.set_title("Comparação da predição da MLP com o valor real do mapa de Hénon\n utilizando a rede neural ótima no conjunto de teste")
ax.set_ylabel('$x[n]$')
ax.set_xlabel('$n$')
ax.set_xlim(4900, 5000)
    
ax.grid(True)
sns.despine()
ax.legend()

plt.show()
fig.savefig("../../../../images/mlp-aprimorada/performance/mlp-aprimorada-vs-henon.png")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

findfont: Font family ['serif'] not found. Falling back to DejaVu Sans.


### Erro Quadrático Médio

In [75]:
from sklearn.metrics import mean_squared_error

In [76]:
mse = mean_squared_error(y_teste, y_pred)
print("MSE = " + str(mse))

MSE = 0.0011025438225542763


### Salvando o Modelo

In [78]:
model.save("../../../../models/mlp-aprimorada/mlp-aprimorada-henon.h5", include_optimizer=True)